In [ ]:
import pandas as pd
import os
import plotly.express as px

In [ ]:
url = r'Z:\HiWi\Popp\TCGA_NSCLC_2022\LUAD\Somatic_mutation\TCGA-LUAD.mutect2_snv.tsv' # r"Z:\HiWi\Popp\TCGA_Breast_2022\TCGA-BRCA.mutect2_snv.tsv" # r'Z:\HiWi\Popp\TCGA_NSCLC_2022\LUAD\Somatic_mutation\TCGA-LUAD.mutect2_snv.tsv'
df_LUAD = pd.read_csv(url, sep='\t', index_col=0)
df_LUAD = df_LUAD[['gene', 'effect']] # kick chr and vaf
df = df_LUAD
df_LUAD
#filter effects missense or all
#include_effect = ['stop_gained', 'stop_lost', 'missense_variant', 'frameshift_variant']
#df = df[df.effect.str.contains('|'.join(include_effect))]
df = df[df.effect == 'missense_variant']
df.reset_index(inplace=True)
df

In [ ]:
# load labels
path = r'D:\FPOPP\MoGCN\result\galant_sweep_14\labels.csv'
folder = os.path.dirname(path)
df_labels = pd.read_csv(path, index_col=1)
df_labels = df_labels[['Sample_ID', 'Labels']]
if 0 in df_labels['Labels']:
    df_labels['Labels'] = df_labels['Labels'] + 1
df_labels

In [ ]:
mutations_list = ['OBSCN', 'KEAP1','FAT3', 'ADAMTS12',  'RP1L1', 'CDH10', 'PAPPA2',
                  'EGFR' ,'STK11','TNR','MUC17', 'RYR3' ,'COL11A1']
df = df[df.gene.isin(mutations_list)]
df

In [ ]:
df_merge = df.merge(df_labels, on='Sample_ID', how='left')
df_merge

In [ ]:
fig = px.bar(df_merge, x='gene', y='effect')
fig.update_xaxes(categoryorder='array', categoryarray= mutations_list)
fig.show()

In [ ]:
# loop through labels and grab top 20 mutations
df_final = pd.DataFrame()
df_group = df_labels.groupby('Labels')
# Loop through each group and plot the scatter plot
for name, df_sub in df_group:
    df_sum = df[df.Sample_ID.isin(df_sub.Sample_ID)]
    counts = df_sum.groupby(['gene', 'effect']).size().reset_index(name='count')
    counts['occurence'] = counts['count'] / len(df_sub.Sample_ID)    
    fig = px.bar(counts, x='gene', y='occurence', color='effect', title=name)
    fig.update_xaxes(categoryorder='array', categoryarray= mutations_list)
    fig.update_layout(yaxis=dict(range=[0,0.4]))
    
    fig.show()